<a href="https://colab.research.google.com/github/hrshtmlng/NLP/blob/main/Emotion_Classification_using_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("emotion")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
# Split
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['validation']['text']
val_labels = dataset['validation']['label']


In [6]:
label_names = dataset['train'].features['label'].names
num_classes = len(label_names)

In [16]:
# Imports
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np

In [8]:
# Tokenizer setup
tokenizer =Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

In [9]:
# Convert texts to sequence
X_train = tokenizer.texts_to_sequences(train_texts)
X_val = tokenizer.texts_to_sequences(val_texts)

In [11]:
# Pad sequences
max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_val = pad_sequences(X_val,maxlen=max_length,padding='pre')

In [12]:
X_val

array([[   0,    0,    0, ...,   10,   13,  533],
       [   0,    0,    0, ..., 1418,   10, 1104],
       [   0,    0,    0, ...,    7,  614,    1],
       ...,
       [   0,    0,    0, ...,    6,  224,  189],
       [   0,    0,    0, ...,   31,   59, 2306],
       [   0,    0,    0, ...,   75,    6,   71]], dtype=int32)

In [13]:
# Labels
y_train = np.array(train_labels)
y_val = np.array(val_labels)


In [17]:
# model
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=max_length),
    GRU(64),
    Dense(num_classes, activation='relu'),
    Dense(num_classes, activation='softmax')

])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=5,batch_size=32)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 52ms/step - accuracy: 0.3389 - loss: 1.5816 - val_accuracy: 0.5970 - val_loss: 1.0127
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.6908 - loss: 0.8460 - val_accuracy: 0.7885 - val_loss: 0.6337
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.8217 - loss: 0.4652 - val_accuracy: 0.8340 - val_loss: 0.4901
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.8689 - loss: 0.3585 - val_accuracy: 0.8565 - val_loss: 0.4371
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9022 - loss: 0.2900 - val_accuracy: 0.8800 - val_loss: 0.4008


In [21]:
def predict_emotion(text):
  seq = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(seq, maxlen=max_length, padding='pre')
  pred = model.predict(padded)[0]
  label = label_names[np.argmax(pred)]
  return label

In [22]:
predict_emotion("I hate everyone!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step


'sadness'

In [25]:
predict_emotion("I'm feeling excited abt my future!" )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


'joy'